In [86]:
# !pip install tensorflow


In [87]:
# !pip install scikit-learn

In [88]:
# !pip install pandas

In [89]:
# !pip install numpy

In [90]:
# !pip install matplotlib

In [91]:
# !pip install --upgrade keras

In [92]:
import wandb
from wandb.integration.kfp import wandb_log

wandb.login()

True

In [93]:
import os
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

In [94]:
import numpy as np
import pandas as pd
from pwnlib.term.readline import history
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.metrics import Precision, Recall
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer # Import Tokenizer from tensorflow.keras.preprocessing.text
from tensorflow.keras.callbacks import TensorBoard
import keras
from keras.models import Model, load_model
from tensorflow.keras.layers import BatchNormalization, Input, Embedding, Dense, Dropout, Flatten, Conv1D, MaxPool1D, Add, Reshape, Concatenate, GlobalMaxPooling1D # Removed 'normalization', 'merge'
from keras.layers import Normalization # Import Normalization from keras.layers
from keras.utils import plot_model, to_categorical # Removed np_utils, to_categorical is sufficient
from keras.preprocessing import sequence
from keras import optimizers

In [95]:
# from __future__ import absolute_import, division, print_function, unicode_literals
# 
# try:
#   # %tensorflow_version only exists in Colab.
#   %tensorflow_version 2.x
# except Exception:
#   pass

In [96]:
# !pip install --upgrade tensorflow


In [97]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [98]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [99]:
pd.set_option("display.max_colwidth", 1024)

In [100]:
# !nvcc --version


In [101]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3034424236916626167
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3760193536
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5518140011491862585
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


I0000 00:00:1730172830.583025    7480 gpu_device.cc:2022] Created device /device:GPU:0 with 3586 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [102]:
tf.debugging.set_log_device_placement(False)

In [103]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [104]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/WAffle/

In [105]:
df = pd.read_csv('Dataset/cisc_database/csic_database.csv')

In [106]:
df.head()

,Unnamed: 0,Method,User-Agent,Pragma,Cache-Control,Accept,Accept-encoding,Accept-charset,language,host,cookie,content-type,connection,lenght,content,classification,URL
0,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.8 (like Gecko),no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=1F767F17239C9B670A39E9B10C3825F4,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/index.jsp HTTP/1.1
1,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.8 (like Gecko),no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=1F767F17239C9B670A39E9B10C3825F4,NaN,close,NaN,NaN,0,http://localhost:8080/favico.ico HTTP/1.1
2,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.8 (like Gecko),no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=81761ACA043B0E6014CA42A4BCD06AB5,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/publico/anadir.jsp?id=3&nombre=Vino+Rioja&precio=100&cantidad=55&B1=A%F1adir+al+carrito HTTP/1.1
3,Normal,POST,Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.8 (like Gecko),no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=933185092E0B668B90676E0A2B0767AF,application/x-www-form-urlencoded,Connection: close,Content-Length: 68,id=3&nombre=Vino+Rioja&precio=100&cantidad=55&B1=A%F1adir+al+carrito,0,http://localhost:8080/tienda1/publico/anadir.jsp HTTP/1.1
4,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.8 (like Gecko),no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=8FA18BA82C5336D03D3A8AFA3E68CBB0,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/publico/autenticar.jsp?modo=entrar&login=choong&pwd=d1se3ci%F3n&remember=off&B1=Entrar HTTP/1.1


In [107]:
df.rename(columns={df.columns[0]: 'Target'}, inplace=True)

In [108]:
def clean_url(url):
    if ' HTTP/' in url:
        return url.split(' HTTP/')[0]
    return url

In [109]:
df['URL'] = df['URL'].apply(clean_url)


In [110]:
df.head()

,Target,Method,User-Agent,Pragma,Cache-Control,Accept,Accept-encoding,Accept-charset,language,host,cookie,content-type,connection,lenght,content,classification,URL
0,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.8 (like Gecko),no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=1F767F17239C9B670A39E9B10C3825F4,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/index.jsp
1,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.8 (like Gecko),no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=1F767F17239C9B670A39E9B10C3825F4,NaN,close,NaN,NaN,0,http://localhost:8080/favico.ico
2,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.8 (like Gecko),no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=81761ACA043B0E6014CA42A4BCD06AB5,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/publico/anadir.jsp?id=3&nombre=Vino+Rioja&precio=100&cantidad=55&B1=A%F1adir+al+carrito
3,Normal,POST,Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.8 (like Gecko),no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=933185092E0B668B90676E0A2B0767AF,application/x-www-form-urlencoded,Connection: close,Content-Length: 68,id=3&nombre=Vino+Rioja&precio=100&cantidad=55&B1=A%F1adir+al+carrito,0,http://localhost:8080/tienda1/publico/anadir.jsp
4,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.8 (like Gecko),no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=8FA18BA82C5336D03D3A8AFA3E68CBB0,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/publico/autenticar.jsp?modo=entrar&login=choong&pwd=d1se3ci%F3n&remember=off&B1=Entrar


In [111]:
df = df[['Target', 'URL']]

In [112]:
print(df['Target'].dtype)


object


In [113]:
label = df['Target'].unique()
print(label)

['Normal' 'Anomalous']


In [114]:
print(df['Target'][0]== 'Normal')

True


In [115]:
# 
# if (df['Target'] == Normal){0} else {1}

# 
df['Target'] = df['Target'].apply(lambda x:0 if str(x) == 'Normal' else 1)
df.head()

,Target,URL
0,0,http://localhost:8080/tienda1/index.jsp
1,0,http://localhost:8080/favico.ico
2,0,http://localhost:8080/tienda1/publico/anadir.jsp?id=3&nombre=Vino+Rioja&precio=100&cantidad=55&B1=A%F1adir+al+carrito
3,0,http://localhost:8080/tienda1/publico/anadir.jsp
4,0,http://localhost:8080/tienda1/publico/autenticar.jsp?modo=entrar&login=choong&pwd=d1se3ci%F3n&remember=off&B1=Entrar


In [116]:
# Training : Validation : Test = 7.5 : 1.5 : 1.0

train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
X_train, X_test, y_train, y_test = train_test_split(df['URL'], df['Target'], test_size=1 - train_ratio, random_state=1110)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=1110)

print(X_train, X_val, X_test)



7707                                                                                                                                                                                                                                                                         http://localhost:8080/tienda1/index.jsp
45711                                                                                                                                                                                                                                                              http://localhost:8080/tienda1/miembros/editar.jsp
932                                                                                                                                                                                                                                                                 http://localhost:8080/tienda1/miembros/salir.jsp
58660                                                                    

In [117]:
print('X_train shape', X_train.shape)
print('y_train shape', y_train.shape)

print('X_val shape', X_val.shape)
print('y_val shape', y_val.shape)

print('X_test shape', X_test.shape)
print('y_test shape', y_test.shape)

X_train shape (45814,)
y_train shape (45814,)
X_val shape (9163,)
y_val shape (9163,)
X_test shape (6109,)
y_test shape (6109,)


In [126]:
test_data = pd.concat([X_test, y_test], axis=1)
test_data = pd.DataFrame(test_data)
test_data.head()

,URL,Target
29533,http://localhost:8080/tienda1/publico/miembros.jsp,0
20641,http://localhost:8080/tienda1/publico/miembros.jsp,0
54451,http://localhost:8080/tienda1/publico/pagar.jsp,1
17286,http://localhost:8080/tienda1/publico/autenticar.jsp?modo=entrar&login=guillory&pwd=Arme%21LlUElA&remember=on&B1=Entrar,0
30944,http://localhost:8080/tienda1/publico/vaciar.jsp,0


In [127]:
# 
test_data.to_csv('Dataset/cisc_database/test_data.csv', index=False)


In [128]:
#URL decode
import urllib.parse
def url_decode(encoded_URL):
  return urllib.parse.unquote(encoded_URL)

In [131]:
def load_data(urls, max_length=1000):
    print(urls[3600])
    urls = [s.lower() for s in urls]
    url_list = []
    for url in urls:
        # url decode
        decoded_url = url_decode(url)
        # unicode encode
        encoded_url = [ord(x) for x in str(decoded_url).strip()]
        encoded_url = encoded_url[:max_length]
        url_len = len(encoded_url)
        if url_len < max_length:
            # zero padding
            encoded_url += ([0] * (max_length - url_len))
        url_list.append((encoded_url))
    # convert to numpy array
    print(urls[3600])
    url_list = np.array(url_list)
    return url_list

In [132]:
train_data = load_data(X_train)
val_data = load_data(X_val)
test_data = load_data(X_test)

http://localhost:8080/tienda1/publico/pagar.jsp?modo=insertar&precio=1599&b1=confirmar%252b
http://localhost:8080/tienda1/publico/pagar.jsp?modo=insertar&precio=1599&b1=confirmar%252b
http://localhost:8080/tienda1/publico/caracteristicas.jsp?ida=2
http://localhost:8080/tienda1/publico/caracteristicas.jsp?ida=2
http://localhost:8080/tienda1/miembros/editar.jsp
http://localhost:8080/tienda1/miembros/editar.jsp


In [123]:
print(train_data)
print(val_data)
print(test_data)

[[104 116 116 ...   0   0   0]
 [104 116 116 ...   0   0   0]
 [104 116 116 ...   0   0   0]
 ...
 [104 116 116 ...   0   0   0]
 [104 116 116 ...   0   0   0]
 [104 116 116 ...   0   0   0]]
[[104 116 116 ...   0   0   0]
 [104 116 116 ...   0   0   0]
 [104 116 116 ...   0   0   0]
 ...
 [104 116 116 ...   0   0   0]
 [104 116 116 ...   0   0   0]
 [104 116 116 ...   0   0   0]]
[[104 116 116 ...   0   0   0]
 [104 116 116 ...   0   0   0]
 [104 116 116 ...   0   0   0]
 ...
 [104 116 116 ...   0   0   0]
 [104 116 116 ...   0   0   0]
 [104 116 116 ...   0   0   0]]


In [39]:
def create_label(labels):
  class_list = [x for x in labels]
  classes = np.array(class_list)
  return classes

In [40]:
train_classes = create_label(y_train)
val_classes = create_label(y_val)
test_classes = create_label(y_test)

In [41]:
print(train_classes)
print(val_classes)
print(test_classes)

[0 1 0 ... 1 0 0]
[1 0 1 ... 0 1 1]
[0 0 1 ... 0 1 0]


## Model Architecture

1. **Input Layer**: 1000 characters
2. **Embedding Layer**: 128 dimensions
3. **Convolution Layer 1**: 
   - Kernel size = K
   - Number of filters = 64
   - Activation = ReLU
4. **Max Pooling Layer 1**: 
   - Kernel size = K
5. **Convolution Layer 2**: 
   - Kernel size = K
   - Number of filters = 64
   - Activation = ReLU
6. **Max Pooling Layer 2**: 
   - Kernel size = size of Convolution Layer 2's output
7. **Concatenate Output & Reshape**: 64-length vector
8. **Fully Connected Layer (FCN)**: 
   - 64 units
   - Activation = ReLU
9. **Batch Normalization**
10. **Dropout**: 0.5
11. **Fully Connected Layer (Output Layer)**: 
   - 1 unit
   - Activation = Sigmoid

**Loss Function**: `binary_crossentropy`


In [42]:
def create_model(input_max_size, embedding_size, kernel_sizes, dropout):

  # Input Layer
  # URLdecode -> Unicode encode -> npumpy.darray
  inputs = Input(shape=(input_max_size,), name='URL_input')

  # Embedding Layer
  x = Embedding(0xffff, embedding_size, name='Embedding')(inputs)
  x = Reshape((input_max_size, embedding_size), name='Reshape_into_128_legnth_vector')(x)

  # Convolution Layers
  convolution_output = []

  for kernel_size in kernel_sizes:
    conv1 = Conv1D(64, kernel_size, activation='relu', padding='same', strides=1)(x)
    pool1 = MaxPool1D(pool_size=kernel_size, padding='same', strides=1)(conv1)
    conv2 = Conv1D(64, kernel_size, activation='relu', padding='same', strides=1)(pool1)
    pool2 = GlobalMaxPooling1D()(conv2)
    convolution_output.append(pool2)

  # concat output
  x = Concatenate(name='Concat_the_outputs')(convolution_output)

  # reshape into 256 length vector
  x = Reshape((256, ), name='Reshape_into_256_length_vector')(x)

  # Fully Connected Layers
  x = Dense(64, activation='relu', name='FullyConnectedLayer')(x)

  # Batch Normalization
  x = BatchNormalization()(x)

  # DropOut
  x = Dropout(dropout)(x)

  # Fully Connected Layers
  predictions = Dense(1, activation='sigmoid', name='Prediction')(x)

  model = Model(inputs=inputs, outputs=predictions, name='Character-level_CNN')

  return model

In [43]:
# def create_model(input_max_size, embedding_size, kernel_size, dropout):
#   
#   # Input Layer
#   inputs = Input(shape=(input_max_size,), name='URL_input')
#   
#   # Embedding Layer
#   x = Embedding(256, embedding_size, name='Embedding')(inputs)
#   x = Reshape((input_max_size, embedding_size), name='Reshape_into_128_length_vector')(x)
#   
#   # Convolution Layer Block
#   conv1 = Conv1D(64, kernel_size, activation='relu', padding='same', strides=1)(x)
#   pool1 = MaxPool1D(pool_size=kernel_size, padding='same', strides=1)(conv1)
#   
#   conv2 = Conv1D(64, kernel_size, activation='relu', padding='same', strides=1)(pool1)
#   pool2 = MaxPool1D(pool_size=pool1.shape[1], padding='same', strides=1)(conv2)
#   
#   # Reshape and Concatenate Output
#   x = Flatten()(pool2)
#   x = Dense(64, activation='relu', name='FullyConnectedLayer')(x)
#   
#   # Batch Normalization
#   x = BatchNormalization()(x)
#   
#   # DropOut
#   x = Dropout(dropout)(x)
#   
#   # Output Layer
#   predictions = Dense(1, activation='sigmoid', name='Prediction')(x)
#   
#   model = Model(inputs=inputs, outputs=predictions, name='Character-level_CNN_Model_A')
#   
#   return model

In [44]:
# config
learning_rate = 0.0042
epochs = 200
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=learning_rate,
#     decay_steps=10000,
#     decay_rate=0.9,
#     staircase=True
# )
lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2, 
    patience=5, 
    min_lr=1e-6)

input_max_size = 1000
embedding_size = 128
kernel_sizes = [4,5,6,7]
# kernel_sizes = 7
dropout = 0.5

loss = 'binary_crossentropy'
optimizer = 'Nadam'

batch_size = 128

# Initialize wandb and pass the variables
wandb.init(project="BaoMatWeb", config={
    "learning_rate": learning_rate,
    "epochs": epochs,
    "input_max_size": input_max_size,
    "embedding_size": embedding_size,
    "kernel_sizes": kernel_sizes,
    "dropout": dropout,
    "loss": loss,
    "optimizer": optimizer,
    "batch_size": batch_size
})




In [45]:
model_filepath = 'saved_model/model' 
checkpoint_filepath = 'logs/checkpoint' 

In [46]:
from tensorflow.keras.callbacks import Callback  # Ensure Callback is imported

class WandbCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Log the metrics to WandB
        wandb.log({
            "val_accuracy": logs.get("val_accuracy"),
            "val_loss": logs.get("val_loss"),
            "val_precision": logs.get("val_precision"),
            "val_recall": logs.get("val_recall")
        })

In [47]:
from tensorflow.keras.callbacks import EarlyStopping

def train(learning_rate, input_max_size, embedding_size, kernel_sizes, dropout, loss, optimizer, train_data, train_classes, epochs, batch_size, val_data, val_classes, model_filepath):

    

    start = learning_rate
    stop = learning_rate * 0.01
    learning_rates = np.linspace(start, stop, epochs)

    model = create_model(input_max_size, embedding_size, kernel_sizes, dropout)
    # Changed 'lr' to 'learning_rate'
    optimizer = optimizers.Nadam(learning_rate=learning_rate, weight_decay= 1e-2, use_ema= True)
    model.compile(loss=loss,
                  optimizer=optimizer,
                  metrics=['accuracy', Precision(), Recall(), 'binary_accuracy'])

    tf_callback = TensorBoard(log_dir="logs", histogram_freq=1)
    # Updated checkpoint_filepath to end with .weights.h5
    checkpoint_filepath = 'logs/checkpoint' + '.weights.h5'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True
        )
    early_stopping = EarlyStopping(
    monitor='val_loss',  # Metric to monitor
    patience=10,          # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

    with tf.device('/GPU:0'):

        history = model.fit(train_data, train_classes,
                  epochs=epochs,
                  batch_size=batch_size,
                  verbose=2,
                  validation_data=(val_data, val_classes),
                  callbacks=[model_checkpoint_callback, WandbCallback(), early_stopping]
                  )

    model.save(model_filepath + '.h5')
    model.save_weights(model_filepath + '.weights.h5')
    return model

In [48]:
saved_model = train(learning_rate, input_max_size, embedding_size, kernel_sizes, dropout, loss, optimizer, train_data, train_classes, epochs, batch_size, val_data, val_classes, model_filepath)

I0000 00:00:1730136443.669121    1477 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3586 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Epoch 1/200


I0000 00:00:1730136447.321813    1655 service.cc:148] XLA service 0x1078a400 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730136447.322211    1655 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-10-29 00:27:27.400371: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1730136447.649067    1655 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-10-29 00:27:34.100613: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng60{k2=0,k5=2,k14=4} for conv (f32[128,128,1,1001]{3,2,1,0}, u8[0]{0}) custom-call(f32[128,64,1,1000]{3,2,1,0}, f32[64,128,1,6]{3,2,1,0}), window={size=1x6 pad=0_0x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","

358/358 - 47s - 132ms/step - accuracy: 0.8261 - binary_accuracy: 0.8261 - loss: 0.3026 - precision: 0.7866 - recall: 0.7918 - val_accuracy: 0.4095 - val_binary_accuracy: 0.4095 - val_loss: 3.0379 - val_precision: 0.4095 - val_recall: 1.0000
Epoch 2/200
358/358 - 21s - 59ms/step - accuracy: 0.8474 - binary_accuracy: 0.8474 - loss: 0.2547 - precision: 0.8118 - recall: 0.8189 - val_accuracy: 0.8519 - val_binary_accuracy: 0.8519 - val_loss: 0.6290 - val_precision: 0.7363 - val_recall: 0.9944
Epoch 3/200
358/358 - 23s - 65ms/step - accuracy: 0.8525 - binary_accuracy: 0.8525 - loss: 0.2439 - precision: 0.8114 - recall: 0.8356 - val_accuracy: 0.8375 - val_binary_accuracy: 0.8375 - val_loss: 0.6030 - val_precision: 0.7162 - val_recall: 0.9989
Epoch 4/200
358/358 - 24s - 66ms/step - accuracy: 0.8498 - binary_accuracy: 0.8498 - loss: 0.2429 - precision: 0.8106 - recall: 0.8282 - val_accuracy: 0.8605 - val_binary_accuracy: 0.8605 - val_loss: 0.2734 - val_precision: 0.7506 - val_recall: 0.9875
Epo

In [49]:
# import plotly.express as px
# # Tạo biểu đồ
# fig = px.line(x=range(len(val_acc)), y=val_acc, labels={'x': 'Epoch', 'y': 'Validation Accuracy'}, title='Validation Accuracy')
# fig.show()

In [50]:
# # Tạo biểu đồ
# fig = px.line(x=range(len(val_loss)), y=val_loss, labels={'x': 'Epoch', 'y': 'Validation Loss'}, title='Validation Loss')
# fig.show()

In [51]:
# import plotly.express as px
# # Tạo biểu đồ
# fig = px.line(x=range(len(val_recall)), y=val_recall, labels={'x': 'Epoch', 'y': 'Validation Recall'}, title='Validation recall')
# fig.show()

In [52]:
# import plotly.express as px
# # Tạo biểu đồ
# fig = px.line(x=range(len(val_precision)), y=val_precision, labels={'x': 'Epoch', 'y': 'Validation Precision'}, title='Validation Precision')
# fig.show()

In [53]:
# import plotly.express as px
# # Tạo biểu đồ
# val_f1 = [2*(a*b)/(a+b) for a,b in zip(val_precision,val_recall)]
# fig = px.line(x=range(len(val_f1)), y=val_f1, labels={'x': 'Epoch', 'y': 'Validation F1'}, title='Validation F1')
# fig.show()

In [54]:
from keras.models import Model, load_model

model_filepath = 'saved_model/model'
model = load_model(model_filepath+'.h5')

In [55]:
import numpy as np
import pandas as pd
from pwnlib.term.readline import history
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.metrics import Precision, Recall
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer # Import Tokenizer from tensorflow.keras.preprocessing.text
from tensorflow.keras.callbacks import TensorBoard
import keras
from keras.models import Model, load_model
from tensorflow.keras.layers import BatchNormalization, Input, Embedding, Dense, Dropout, Flatten, Conv1D, MaxPool1D, Add, Reshape, Concatenate, GlobalMaxPooling1D # Removed 'normalization', 'merge'
from keras.layers import Normalization # Import Normalization from keras.layers
from keras.utils import plot_model, to_categorical # Removed np_utils, to_categorical is sufficient
from keras.preprocessing import sequence
from keras import optimizers

def load_data(urls, max_length=1000):
    urls = [s.lower() for s in urls]
    url_list = []
    for url in urls:
        # url decode
        decoded_url = url_decode(url)
        # unicode encode
        encoded_url = [ord(x) for x in str(decoded_url).strip()]
        encoded_url = encoded_url[:max_length]
        url_len = len(encoded_url)
        if url_len < max_length:
            # zero padding
            encoded_url += ([0] * (max_length - url_len))
        url_list.append((encoded_url))
    # convert to numpy array
    url_list = np.array(url_list)
    return url_list

url = ["http://localhost:8080/sqli.php?input=%27or+1%3D1%3B--","http://127.0.0.1:8080/favico.ico", "http://localhost:8080/tienda1/login/index.php","http://localhost:8080/tienda1.BAK", "http://localhost:8080/tienda1/publico/autenticar.jsp", "http://localhost:8080/tienda1/miembros/imagenes/zarauz.jpg"]

prediction = model.predict(load_data(url))



print(prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 810ms/step
[[0.99979323]
 [0.00670491]
 [0.12484346]
 [0.9996044 ]
 [0.5814888 ]
 [0.00135562]]


In [56]:
results = model.evaluate(test_data, test_classes, verbose=2)
test_loss = results[0]
test_accuracy = results[1]  # Adjust based on metric order
test_precision = results[2]
test_recall = results[3]


wandb.log({
            "test_loss": test_loss,
            "test_accuracy": test_accuracy,
            "test_precision":test_precision,
            "test_recall": test_recall
        })


191/191 - 3s - 14ms/step - accuracy: 0.8623 - binary_accuracy: 0.8623 - loss: 0.2194 - precision: 0.7681 - recall: 0.9454


In [57]:
import wandb
wandb.finish()

test_accuracy,▁
test_loss,▁
test_precision,▁
test_recall,▁
val_accuracy,▁█████████████▇██████▇█▇██▇███▇▇▇███▇█
val_loss,█▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁
val_precision,▁▅▅▅▅▅▅▅▅▅▅▇▅▅█▅▅▅▅▆▅█▅█▅▅█▅▅▅███▅▅▅██
val_recall,███████████▃██▁████▆█▁█▁▇█▁▇██▁▁▁███▁▁
test_accuracy,0.86233
test_loss,0.21945
test_precision,0.76814
